The purpose of this jupyter notebook is to scrape the Billboard Top 100 charts  song, artist. To do this, I will be using BeautifulSoup.



In [31]:
import re 
from sklearn import datasets 
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import random
import requests
from bs4 import BeautifulSoup
import json
import numpy as np
import pandas as pd




#print('Welcome!')
#input_song = input("Please select the title of the song: ")

#input_artist = input("Please select the title of the artist: ")


In [32]:
url = 'https://www.billboard.com/charts/hot-100/'
    
def billboard_web_scrape(url):
    top_100 = {'title':[], 'artist':[]}
  
    page = requests.get(url)
        
    # Parse HTML
    soup = BeautifulSoup(page.content, 'html.parser')
        
    # Grab relevant info from top 100 charts 
    for elem in soup.find_all("li",{'class':'chart-list__element display--flex'}):
        title = elem.find_all("span",{'class':'chart-element__information__song text--truncate color--primary'})[0].get_text()
        artist = elem.find_all("span",{'class':'chart-element__information__artist text--truncate color--secondary'})[0].get_text()
        top_100['title'].append(title)
        top_100['artist'].append(artist)
    #for minisoup in soup.find_all(class_='chart-list-item'):
    #    print(minisoup)
       
        #song['title'] = minisoup['data-title']
        #song['artist'] = minisoup['data-artist']
        #try: 
        #    song['title'] = minisoup.find('span', {'class':'chart-element__information__song text--truncate color--primary'}).get_text(strip=True)
                
        #except:
        #    song['title'] = np.nan
        #try: 
        #    song['artist'] = minisoup.find('span', {'chart-element__information__artist text--truncate color--secondary'}).get_text(strip=True)
        #except:
        #    song['artist'] = np.nan
        #top_100.append(song)
    return pd.DataFrame(top_100)


top_100 = billboard_web_scrape('https://www.billboard.com/charts/hot-100/')
top_100['orig'] = 1
top_100.head()

,title,artist,orig
0,Butter,BTS,1
1,Good 4 U,Olivia Rodrigo,1
2,Levitating,Dua Lipa Featuring DaBaby,1
3,Peaches,Justin Bieber Featuring Daniel Caesar & Giveon,1
4,Leave The Door Open,Silk Sonic (Bruno Mars & Anderson .Paak),1


In [33]:
#pip install spotipy
!pip install spotipy


import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
cid = 'a5e6ae05f5e4482dab95cc0c1c8b3b42'
secret = '6b7fd51300c84b808ecc013d38a47112'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager
=
client_credentials_manager)





In [57]:
def get_playlist_tracks(username, playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

#Playlist Spotify Songtitles & artists
all_tracks = get_playlist_tracks("spotify", "6yPiKpy7evrwvZodByKvM9")

print(all_tracks)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [63]:
title = [all_tracks[i]["track"]["name"] for i in range(len(all_tracks))]
artist = [all_tracks[i]["track"]["artists"][0]["name"] for i in range(len(all_tracks))]
spotify_data = pd.DataFrame(list(zip(title, artist)),
              columns=['title','artist'])


spotify_data['orig'] = 2

spotify_data.head(300)


,title,artist,orig
0,2K,Nosaj Thing,2
1,4 Billion Souls,The Doors,2
2,4 Minute Warning,Radiohead,2
3,7 Element,Vitas,2
4,#9 Dream,R.E.M.,2
...,...,...,...
295,Black Widow,Cage The Elephant,2
296,Blackbird - Anthology 3 Version,The Beatles,2
297,Blackbird - Remastered 2009,The Beatles,2
298,Blackbird / Yesterday,The Beatles,2


In [64]:
#spotify_billboard_short = spotify_billboard.drop([song_titles','song_artists'], axis=1)

#List of all billboard and spotify together
spotify_billboard = pd.concat([top_100, spotify_data])
spotify_billboard.reset_index(inplace=True)

spotify_billboard.head()

,index,title,artist,orig
0,0,Butter,BTS,1
1,1,Good 4 U,Olivia Rodrigo,1
2,2,Levitating,Dua Lipa Featuring DaBaby,1
3,3,Peaches,Justin Bieber Featuring Daniel Caesar & Giveon,1
4,4,Leave The Door Open,Silk Sonic (Bruno Mars & Anderson .Paak),1


In [86]:

def get_uri():
        uri_list = []
        for song in spotify_billboard["title"]:
            song_response = sp.search(q=userinput, limit=1) 
            uri = sp.audio_features(song["tracks"]["items"][0]["uri"])[0]
            uri_list.append(uri)          
        spotify_billboard["uri"] = uri.list
        return spotify_billboard
    
 print(spotify_billboard)
        


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 10)

In [60]:
#scaling the whole list
  scaler = StandardScaler()
  scaler.fit(spotify_billboard)
  new_scaled = scaler.transform(spotify_billboard)
  new_df = pd.DataFrame(new_scaled)
  


    #clustering everything
  kmeans = KMeans(n_clusters=12, random_state=1234)
  kmeans.fit(new_df.iloc[:,:-1])

  clusters = kmeans.predict(new_df.iloc[:,:-1])
  pd.Series(clusters).value_counts().sort_index()
  #Final clustering
  all_scaled_df = pd.DataFrame(new_df)
  all_scaled_df["cluster"] = clusters
    
    
    
import random
if input_song in spotify_billboard[spotify_billboard['orig'] == 1]['title'].values: 
  cluster_general = all_scaled_df[all_scaled_df['cluster'] == (list(all_scaled_df["cluster"])[-1])]
  double_index = spotify_billboard[:-1]["orig"][spotify_billboard["orig"] == 1] 
  sample_ids = list(cluster_general)



else
  print("This song is not in the top 100 but I've got another option for you")
  cluster_general = all_scaled_df[all_scaled_df['cluster'] == (list(all_scaled_df["cluster"])[-1])]
  sample_ids = list(cluster_general)
  print(cluster_general)
  try:
    double_index = spotify_billboard[:1]["orig"][spotify_billboard["orig"] == 2]
  except:
    pass

  #print(sample_ids)
  suggestion = random.choice(sample_ids)
  print(suggestion)
  print(spotify_billboard.at[suggestion, "title"] + ", by: " + spotify_billboard.at[suggestion, "artist"])   







KeyError: 0